In [103]:
import pandas as pd
import numpy as np

In [174]:
pandemic = pd.read_csv("final/Confirmed cases and deaths.csv")
prepandemic = pd.read_csv("final/prepandemic_v2.csv")
unemployment = pd.read_csv("final/Unemployment_v1.csv")

# Remove all Puerto Rico data
pandemic = pandemic.drop(pandemic[pandemic['stname'] == 'Puerto Rico'].index)

/Users/h0y01c9/opt/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (309,312,314,317,329,332,334) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [177]:
# Need state and county name
state = pandemic['stname']
county = pandemic['ctyname']

# The confirmed cases and death number is accumulated, used the last available date data
confirmed_cases = pandemic['confirmed_cases_20200502']
deaths = pandemic['death20200502']

In [178]:
# Get days between first confirmed case to now
import datetime
def days(pandemic_row, start = datetime.datetime(2020,1,22), end = datetime.datetime(2020,4,18), min_number = 200, min_precent=0.3):
    end_title = 'confirmed_cases_' +  end.strftime("%Y%m%d")
    total_days = (end - start).days + 1
    min_cases = min(pandemic_row[end_title] * min_precent, min_number)
    current = start
    accumulate_cases = 0
    days = 0
    while current <= end:
        title = 'confirmed_cases_' +  current.strftime("%Y%m%d")
        accumulate_cases += pandemic_row[title]
        if accumulate_cases < min_cases:
            days += 1
        else:
            break

        current += datetime.timedelta(days=1)
    
    return total_days - days
case_days = pandemic.apply(lambda x: days(x,end = datetime.datetime(2020,5,2)), axis = 1) 
case_days

0        37
1        35
2        25
3        30
4        36
5        36
6        22
7        34
8        35
9        35
10       36
11       31
12       28
13       35
14       37
15       26
16       33
17       29
18       35
19       31
20       31
21       38
22       27
23       32
24       31
25       42
26       30
27       32
28       33
29       30
       ... 
3112     17
3113     41
3114     41
3115     36
3116     27
3117     44
3118     48
3119     37
3120     21
3121     37
3122     42
3123     31
3124     26
3125     43
3126     36
3127     22
3128     37
3129     39
3130     28
3131     38
3132     23
3133     46
3134    102
3135     48
3136     36
3137     34
3138     37
3139     29
3140     35
3141    102
Length: 3142, dtype: int64

In [179]:
# Use year 2018(closest) population density
populationdensity=prepandemic['popdensity_2018']
populationdensity_min = populationdensity.min()
populationdensity_mean = populationdensity.mean()
populationdensity_stage = populationdensity_mean - populationdensity_min
def populationdensity_category(populationdensity):
    if populationdensity < populationdensity_mean - populationdensity_stage * 0.9: 
        return '1_Very Low' 
    elif populationdensity < populationdensity_mean - populationdensity_stage * 0.3: 
        return '2_Low'  
    elif populationdensity < populationdensity_mean + populationdensity_stage * 0.3: 
        return '3_Medium'   
    elif populationdensity < populationdensity_mean + populationdensity_stage: 
        return '4_High' 
    else:
        return 'Very_High'
populationdensity_category = prepandemic.apply(lambda x: populationdensity_category(x['popdensity_2018']), axis = 1)

In [181]:
# Get top 3 most populations ethnicity group in the county
def ethnic_group_top(hispanic, white, black, indian, asian, hawaii_na, top):
    ethnic_groups = [('Hispanic', hispanic), ('White', white), ('Black', black), ('Indian', indian), ('Asian', asian), ('Hawaii and N/A', hawaii_na)]
    ethnic_groups = sorted(ethnic_groups, key = lambda x: -x[1])
    return ethnic_groups[top - 1][0]

ethnic_group_top1 = prepandemic.apply(lambda x: ethnic_group_top(x['hispanic_2018'], x['nhwhite_2018'], x['nhblack_2018'], x['nhindian_2018'], x['nhasian_2018'], x['nhhawaii_2018'], 1), axis = 1)
ethnic_group_top2 = prepandemic.apply(lambda x: ethnic_group_top(x['hispanic_2018'], x['nhwhite_2018'], x['nhblack_2018'], x['nhindian_2018'], x['nhasian_2018'], x['nhhawaii_2018'], 2), axis = 1)
ethnic_group_top3 = prepandemic.apply(lambda x: ethnic_group_top(x['hispanic_2018'], x['nhwhite_2018'], x['nhblack_2018'], x['nhindian_2018'], x['nhasian_2018'], x['nhhawaii_2018'], 3), axis = 1)

In [182]:
# Age data has 18 columns, we group them to children, young adult, middle adult, senior
# column1:  age 0-4 prop.
# column2:  age 5-9 prop.
# column3:  age 10-14 prop.
# column4:  age 15-19 prop.
# column5:  age 20-24 prop.
# column6:  age 25-29 prop.
# column7:  age 30-34 prop.
# column8:  age 35-39 prop.
# column9:  age 40-44 prop.
# column10: age 45-49 prop.
# column11: age 50-54 prop.
# column12: age 55-59 prop.
# column13: age 60-64 prop.
# column14: age 65-69 prop.
# column15: age 70-74 prop.
# column16: age 75-79 prop.
# column17: age 80-84 prop.
# column18: age 85+ prop.
def age_group_top(age, top):
    children_poportion = age[0] + age[1] + age[2] + age[3]
    young_adult_poportion = age[4] + age[5] + age[6] + age[7] + age[8]
    middle_adult_poportion = age[9] + age[10] + age[11] + age[12]
    senior_poportion = age[13] + age[14] + age[15] + age[16] + age[17]
    age_group_list = [('Children', children_poportion), ('Young Adult', young_adult_poportion), ('Middle Adult', middle_adult_poportion), ('Senior', senior_poportion)]
    age_group_list = sorted(age_group_list, key = lambda x: -x[1])
    return age_group_list[top - 1][0]
age_groups_top1 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 1), axis = 1)
age_groups_top2 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 2), axis = 1)
age_groups_top3 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 3), axis = 1)
age_groups_top4 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 4), axis = 1)

In [183]:
# Get all education adata
education = pd.read_excel("final/Education_By_County.xls")
education = education.drop(education[pd.isna(education['2013 Rural-urban Continuum Code'])].index)

In [184]:
states_map = pd.read_csv("final/states.csv")
states_map

for index, row in states_map.iterrows():
    pc = len(pandemic[pandemic['stname'] == row[1]])
    ec = len(education[education['State'] == row[0]])
    if pc != ec:
        print(f"In education data set[{row[0]}]: {ec} rows, in pandemic dataset[{row[1]}]: {pc} rows")  

In education data set[VA]: 134 rows, in pandemic dataset[Virginia]: 133 rows
In education data set[PR]: 78 rows, in pandemic dataset[Puerto Rico]: 0 rows


In [185]:
va_pandemic = pandemic[pandemic['stname'] == 'Virginia']
va_education = education[education['State'] == 'VA']
for index, row in va_education.iterrows():
    area = row['Area name']
    if len(va_pandemic[va_pandemic['ctyname'] == area]) == 0:
        print(f"{area} is missing in va_pandemic")

Bedford city is missing in va_pandemic


In [186]:
education = education.drop(education[education['Area name'] == 'Bedford city'].index)

#education['Percent of adults completing some college or associate\'s degree, 2014-18']
# education['Percent of adults with a bachelor\'s degree or higher, 2014-18']

# Age data has 18 columns, we group them to children, young adult, middle adult, senior
# column1:  less than high school diploma poportion.
# column2:  high school diploma only poportion.
# column3:  some college or associate degress poportion.
# column4:  bachelor or higher education poportion.
def education_group_top(less_than_high_school, high_school, college, bachelor_above, top):
    education_group = [('Less_than_high_school', less_than_high_school), ('High_school', high_school), ('Some_college_or_associate', college), ('Bachelor_Or_Above', bachelor_above)]
    education_group = sorted(education_group, key = lambda x: -x[1])
    return education_group[top - 1][0]

education_group_top1 = education.apply(lambda x: education_group_top(x['Percent of adults with less than a high school diploma, 2014-18'], 
                                                                     x['Percent of adults with a high school diploma only, 2014-18'], 
                                                                     x['Percent of adults completing some college or associate\'s degree, 2014-18'], 
                                                                     x['Percent of adults with a bachelor\'s degree or higher, 2014-18'], 1), axis=1)
education_group_top2 = education.apply(lambda x: education_group_top(x['Percent of adults with less than a high school diploma, 2014-18'], 
                                                                     x['Percent of adults with a high school diploma only, 2014-18'], 
                                                                     x['Percent of adults completing some college or associate\'s degree, 2014-18'], 
                                                                     x['Percent of adults with a bachelor\'s degree or higher, 2014-18'], 2), axis=1)
education_group_top3 = education.apply(lambda x: education_group_top(x['Percent of adults with less than a high school diploma, 2014-18'], 
                                                                     x['Percent of adults with a high school diploma only, 2014-18'], 
                                                                     x['Percent of adults completing some college or associate\'s degree, 2014-18'], 
                                                                     x['Percent of adults with a bachelor\'s degree or higher, 2014-18'], 3), axis=1)
education_group_top4 = education.apply(lambda x: education_group_top(x['Percent of adults with less than a high school diploma, 2014-18'], 
                                                                     x['Percent of adults with a high school diploma only, 2014-18'], 
                                                                     x['Percent of adults completing some college or associate\'s degree, 2014-18'], 
                                                                     x['Percent of adults with a bachelor\'s degree or higher, 2014-18'], 4), axis=1)

In [187]:
populations = pd.read_excel("final/co-est2019-annres.xlsx")
populations['county'] = populations['Geographic Area'].apply(lambda x: x.split(',')[0][1:])
populations['state'] = populations['Geographic Area'].apply(lambda x: x.split(',')[1])
population = populations['Census']

In [189]:
result = pd.concat([state, county], axis=1)
                    
result['confirmed_cases'] = confirmed_cases
result['deaths'] = deaths
result['population'] = population
result['days'] = case_days
result['avearge_case'] = (100000.0) * confirmed_cases / case_days / population # Daily confirmed case per 100k people
result['populationdensity'] = populationdensity_category
result['toppest_ethnicity'] = ethnic_group_top1
result['second_ethnicity'] = ethnic_group_top2
result['third_ethnicity'] = ethnic_group_top3
result['toppest_age_group'] = age_groups_top1
result['second_age_group'] = age_groups_top2
result['third_age_group'] = age_groups_top3
result['fourth_age_group'] = age_groups_top4
result['top_education'] = education_group_top1
result['second_top_education'] = education_group_top2
result['popdensity'] = prepandemic['popdensity_2018']
result['precent_Hispanic'] = prepandemic['hispanic_2018'] 
result['precent_White'] = prepandemic['nhwhite_2018'] 
result['precent_Black'] = prepandemic['nhblack_2018'] 
result['precent_Idian'] = prepandemic['nhindian_2018'] 
result['precent_Asian'] = prepandemic['nhasian_2018'] 
result['precent_Hawaii'] = prepandemic['nhhawaii_2018']
result['bachelor'] = education['Percent of adults with a bachelor\'s degree or higher, 2014-18']
result['college'] = education['Percent of adults completing some college or associate\'s degree, 2014-18']
result['highschool'] = education['Percent of adults with a high school diploma only, 2014-18']
result['lessthanhighschool'] = education['Percent of adults with less than a high school diploma, 2014-18']
result['precent_under_20'] = prepandemic['ageg1_2018'] + prepandemic['ageg2_2018'] + prepandemic['ageg3_2018'] + prepandemic['ageg4_2018']
result['precent_20_to_40'] = prepandemic['ageg5_2018'] + prepandemic['ageg6_2018'] + prepandemic['ageg7_2018'] + prepandemic['ageg8_2018'] + prepandemic['ageg9_2018']
result['precent_40_to_60'] = prepandemic['ageg10_2018'] + prepandemic['ageg11_2018'] + prepandemic['ageg12_2018'] + prepandemic['ageg13_2018']
result['precent_above_60'] = prepandemic['ageg14_2018'] + prepandemic['ageg15_2018'] + prepandemic['ageg16_2018'] + prepandemic['ageg17_2018'] + prepandemic['ageg18_2018']

In [190]:
# Target is 5 groups, very low, low, medium, high and very high
def target_group(rank, total):
    if (rank < total / 5.0):
        return "Very_Low"
    if (rank < total * 2.0 / 5.0):
        return "Low"
    if (rank < total * 3.0 / 5.0):
        return "Medium"
    if (rank < total * 4.0 / 5.0):
        return "High"
    return "Very_High"

# Binary Target only has low and high
def binary_target_group(rank, total):
    if (rank < total / 2.0):
        return "Low"
    return "High"

total = len(result)

result['target_rank'] = result['avearge_case'].rank().apply(lambda r: target_group(r, total))
result['binary_target_rank'] = result['avearge_case'].rank().apply(lambda r: binary_target_group(r, total))

In [191]:
result.to_csv('final/model.csv')

In [118]:
y = result['avearge_case']

In [127]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

In [132]:
label_encoder = LabelEncoder()
populationdensity_onehot = label_encoder.fit_transform(populationdensity_category)
ethnic_group_top1_onehot = label_encoder.fit_transform(ethnic_group_top1)
ethnic_group_top2_onehot = label_encoder.fit_transform(ethnic_group_top2)
ethnic_group_top3_onehot = label_encoder.fit_transform(ethnic_group_top3)

age_groups_top1_onehot = label_encoder.fit_transform(age_groups_top1)
age_groups_top2_onehot = label_encoder.fit_transform(age_groups_top2)
age_groups_top3_onehot = label_encoder.fit_transform(age_groups_top3)
age_groups_top4_onehot = label_encoder.fit_transform(age_groups_top4)
education_group_top1_onehot = label_encoder.fit_transform(education_group_top1)
education_group_top2_onehot = label_encoder.fit_transform(education_group_top2)

2

In [125]:
mutual_info_classif(X, y, discrete_features=True)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').